# client instance 생성

In [1]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
# DEPLOYMENT_URL="http://localhost:2024/"
# DEPLOYMENT_URL="http://localhost:8000/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [2]:
def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data
 
 
def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)
 
        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    if msg_type == "ai":
                        return True
    return False

In [3]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': '2d00752a-ac6c-4418-a68a-b4dfbffd02fd', 'created_at': '2025-08-18T05:11:43.051443+00:00', 'updated_at': '2025-08-18T05:11:43.051443+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None, 'interrupts': {}, 'error': None}
2d00752a-ac6c-4418-a68a-b4dfbffd02fd


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [4]:
list_msg_id = list()


def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data


def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)

        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    msg_id = data['messages'][-1]['id']
                    if msg_id not in list_msg_id and msg_type == "ai":
                        list_msg_id.append(msg_id)
                        return True
    return False


def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True

In [5]:
assistant_id = "slide_generate_agent"
list_chunk = list()

In [6]:
input = { "messages": "제주도 여행"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    list_chunk.append(chunk)
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])
    # elif filter_silde_data(chunk):
    #     print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
    #     print(chunk)


##########  AI 메시지  ##########
제주도 여행 슬라이드을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

1. 타이틀 페이지
2. 목차 페이지
3. 제주도 여행 일정 제안 - 제주도를 여행하기 위한 최적의 일정과 코스를 제안합니다. 여행을 원활하게 계획하고, 중요한 명소들을 놓치지 않도록 도와줍니다. 
4. 제주도 관광 명소 - 제주도의 필수 관광 명소와 각 명소의 개요를 제공합니다. 방문자들이 가장 많이 찾는 유명한 장소들과 그 외 숨겨진 비경을 소개합니다. 
5. 제주도 맛집 - 제주에서 반드시 방문해야 할 맛집을 소개합니다. 현지의 추천을 통해 가장 인기 있는 식당과 미식을 경험할 수 있는 장소를 알려드립니다. 
6. 제주도의 숙소 추천 - 제주도의 다양한 숙소 옵션을 비교하고 추천해드립니다. 호텔, 리조트, 펜션 등 저렴하면서도 편안하게 머물 수 있는 장소를 확인하세요. 
7. 제주도 문화체험 - 제주도에서 특별한 문화 체험 기회를 소개합니다. 제주만의 독특한 문화와 역사적 기반을 직접 경험할 수 있는 활동과 장소를 소개합니다. 

##########  AI 메시지  ##########

    슬라이드 생성에 필요한 조사를 시작합니다.
    
##########  AI 메시지  ##########

    슬라이드에 사용할 이미지를 검색하고 있습니다.
    
##########  AI 메시지  ##########

    타이틀 페이지에 사용할 이미지를 검색하였습니다.
    
##########  AI 메시지  ##########

    타이틀 페이지에 대한 조사를 완료하였습니다.
    
##########  AI 메시지  ##########

    슬라이드 디자인을 시작합니다.
    
##########  AI 메시지

In [7]:
for chunk in list_chunk:
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])


In [8]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] != "human":
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        print(msg["content"])

##########  AI 메시지  ##########
제주도 여행 슬라이드을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

1. 타이틀 페이지
2. 목차 페이지
3. 제주도 여행 일정 제안 - 제주도를 여행하기 위한 최적의 일정과 코스를 제안합니다. 여행을 원활하게 계획하고, 중요한 명소들을 놓치지 않도록 도와줍니다. 
4. 제주도 관광 명소 - 제주도의 필수 관광 명소와 각 명소의 개요를 제공합니다. 방문자들이 가장 많이 찾는 유명한 장소들과 그 외 숨겨진 비경을 소개합니다. 
5. 제주도 맛집 - 제주에서 반드시 방문해야 할 맛집을 소개합니다. 현지의 추천을 통해 가장 인기 있는 식당과 미식을 경험할 수 있는 장소를 알려드립니다. 
6. 제주도의 숙소 추천 - 제주도의 다양한 숙소 옵션을 비교하고 추천해드립니다. 호텔, 리조트, 펜션 등 저렴하면서도 편안하게 머물 수 있는 장소를 확인하세요. 
7. 제주도 문화체험 - 제주도에서 특별한 문화 체험 기회를 소개합니다. 제주만의 독특한 문화와 역사적 기반을 직접 경험할 수 있는 활동과 장소를 소개합니다. 

##########  AI 메시지  ##########

    슬라이드 생성에 필요한 조사를 시작합니다.
    
##########  AI 메시지  ##########

    슬라이드에 사용할 이미지를 검색하고 있습니다.
    
##########  AI 메시지  ##########

    제주도 문화체험 페이지에 사용할 이미지를 검색하였습니다.
    
##########  AI 메시지  ##########

    제주도 문화체험 페이지에 대한 조사를 완료하였습니다.
    
##########  AI 메시지  ##########

    슬라이드 디자인을 시작합니다.
    
########

In [9]:
len(state["values"]["slides"])

7

In [10]:
state["values"]["slides"]

[{'description': '제주도 여행 슬라이드에 대한 프레젠테이션 타이틀 페이지',
  'requirements': ['타이틀 텍스트', '부제목 또는 설명'],
  'research': True,
  'slide_type': 'title',
  'topic': '제주도 여행 슬라이드',
  'idx': 0,
  'name': '타이틀',
  'content': "## 제주도 여행 슬라이드 타이틀\n\n제주도는 한국의 대표적인 휴양지로, 국내뿐 아니라 해외에서도 많은 관광객들이 찾는 아름다운 섬입니다. 제주도를 주제로 한 프레젠테이션은 그 매력과 아이콘을 중심으로 구성되어야 합니다. 타이틀 페이지는 발표의 첫인상을 결정하는 중요한 요소로, 제주도의 독특한 자연경관과 문화적 아이콘을 효과적으로 담아내야 합니다.\n\n제주도를 대표하는 이미지로는 화산지형의 아름다움과 한라산, 성산 일출봉, 그리고 유명한 돌하르방이 있습니다. 제주도의 풍부한 자연경관은 관광객들에게 다양한 활동을 제공합니다. 생동감 넘치는 들판과 해변은 휴식과 트레킹, 그리고 각종 야외 활동에 적합합니다. 이러한 이미지와 개념을 슬라이드 타이틀에 적절히 배치하면, 청중에게 제주도의 매력을 강렬히 전달할 수 있습니다[^1^].\n\n또한 제주도는 '한국의 하와이'로 알려져 있으며, 유네스코 세계유산으로 등록된 곳이기도 합니다. 그 독특한 문화와 경험은 제주도 여행을 더욱 특별하게 만듭니다. 해녀 문화, 감귤 농장, 독특한 지역 요리 등을 통해 한층 더 높은 관심을 유도할 수 있습니다. 이러한 요소는 프레젠테이션의 첫 페이지에 배경 이미지나 키비주얼로 활용할 수 있습니다[^2^][^3^].\n\n타이틀 디자인은 깔끔하면서 전문적으로 보이는 것이 중요합니다. 제주도의 테마를 살린 색상과 간결한 폰트를 사용함으로써 보다 효과적으로 시각적인 전달이 가능합니다. 예를 들어 하늘색이나 바닷빛을 반영하는 일러스트를 사용하여 여행의 느낌을 강화할 수 있습니다[^4^][^5^]. 이러한 디자인적 요소는 프레젠

In [11]:
for s in state["values"]["slides"]:
    with open(f"{s['idx']}.html", "w", encoding="utf-8") as f:
        f.write(s['html'])